In [ ]:
import os, sys

print(sys.executable)
os.environ["python_bin"] = sys.executable

In [ ]:
!$python_bin -m pip install -r requirements.txt

In [ ]:
base = input("URL: ")
regex = input("regex: ")

In [ ]:
import aiohttp
import asyncio
import logging
from scraper import WebScraper

logging.basicConfig(level="INFO")


async with aiohttp.ClientSession() as session:
    scraper = WebScraper(base, regex, session)
    await scraper.scrape()

In [ ]:
from urllib.parse import urlparse
import re


def get_groups(urls):
    parsed = list(map(urlparse, urls))
    groups = list(
        set(
            map(
                lambda u: re.sub(r"\d+\.jpg", "", u.path.split("/")[-1]),
                parsed_collected,
            )
        )
    )
    return groups


groups = get_groups(scraper.collected)
group = groups[3]
urls = [url for url in scraper.collected if group in url]

display(urls)

In [ ]:
# https://stackoverflow.com/a/32370538
from IPython.display import Image

for img in urls[:2]:
    # https://stackoverflow.com/q/19471814
    display(Image(url=img))